# 03. 합성곱 신경망 - MNIST (CNN)

이미지 분류에 특화된 CNN을 구현합니다.

## 학습 목표
- 합성곱(Convolution) 연산 이해
- 풀링(Pooling) 연산 이해
- CNN 구조 설계
- MNIST 분류

In [ ]:
import torch
import matplotlib.pyplot as plt

torch.manual_seed(42)

## 1. 합성곱 연산 이해

합성곱은 필터(커널)를 이미지 위에서 슬라이딩하며 특징을 추출합니다.

In [ ]:
from mlfs.nn.layers import Conv2d, MaxPool2d

# 예시: 단일 이미지에 합성곱 적용
conv = Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)
pool = MaxPool2d(kernel_size=2)

# 입력: (batch=1, channels=1, H=28, W=28)
x = torch.randn(1, 1, 28, 28)

# 합성곱 적용
conv_out = conv(x)
print(f"Input: {x.shape}")
print(f"After Conv: {conv_out.shape}")

# 풀링 적용
pool_out = pool(conv_out)
print(f"After Pool: {pool_out.shape}")

## 2. MNIST 데이터 로드

In [ ]:
from mlfs.utils.data import load_mnist
from mlfs.utils.viz import plot_images

# 이미지 형태로 로드 (flatten=False)
X_train, y_train = load_mnist(train=True, flatten=False)
X_test, y_test = load_mnist(train=False, flatten=False)

print(f"Train: {X_train.shape}")
print(f"Test: {X_test.shape}")

# 샘플 확인
plot_images(X_train[:10], labels=y_train[:10])

## 3. CNN 모델

In [ ]:
from mlfs.nn.models import CNN
from mlfs.nn.losses import CrossEntropyLoss
from mlfs.nn.optim import Adam

# CNN 모델 생성
model = CNN(in_channels=1, num_classes=10, image_size=28)
optimizer = Adam(model.parameters(), lr=0.001)
criterion = CrossEntropyLoss()

print(model)

In [ ]:
# 학습 (작은 서브셋으로 빠른 테스트)
n_samples = 10000  # 빠른 학습을 위해 일부만 사용
X_train_sub = X_train[:n_samples]
y_train_sub = y_train[:n_samples]

epochs = 10
batch_size = 64

train_losses = []
test_accs = []

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    
    indices = torch.randperm(n_samples)
    
    for i in range(0, n_samples, batch_size):
        batch_idx = indices[i:i+batch_size]
        X_batch = X_train_sub[batch_idx]
        y_batch = y_train_sub[batch_idx]
        
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    train_losses.append(epoch_loss / (n_samples // batch_size))
    
    # 테스트
    model.eval()
    with torch.no_grad():
        # 배치로 테스트 (메모리 절약)
        correct = 0
        for i in range(0, len(X_test), batch_size):
            X_batch = X_test[i:i+batch_size]
            y_batch = y_test[i:i+batch_size]
            pred = model.predict(X_batch)
            correct += (pred == y_batch).sum().item()
        test_acc = correct / len(X_test)
    test_accs.append(test_acc)
    
    print(f"Epoch {epoch + 1:2d}: Loss = {train_losses[-1]:.4f}, Test Acc = {test_acc:.4f}")

In [ ]:
# 결과 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(train_losses)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(test_accs)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Test Accuracy')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 예측 결과 시각화
model.eval()
with torch.no_grad():
    predictions = model.predict(X_test[:20])

plot_images(X_test[:20], labels=y_test[:20], predictions=predictions, n_rows=4, n_cols=5)

## 4. 필터 시각화

학습된 첫 번째 합성곱 레이어의 필터를 확인합니다.

In [ ]:
# 첫 번째 Conv 레이어의 가중치
filters = model.conv1.weight.detach()
print(f"Filter shape: {filters.shape}")

# 필터 시각화
fig, axes = plt.subplots(4, 8, figsize=(12, 6))
for i, ax in enumerate(axes.flatten()):
    if i < filters.shape[0]:
        ax.imshow(filters[i, 0], cmap='gray')
    ax.axis('off')
plt.suptitle('Learned Filters (Conv1)', fontsize=14)
plt.tight_layout()
plt.show()

## 요약

1. **합성곱층**: 지역적 특징 추출, 파라미터 공유
2. **풀링층**: 공간 크기 축소, 위치 불변성
3. **CNN 구조**: Conv → ReLU → Pool → ... → FC
4. **성능**: MLP보다 적은 파라미터로 더 좋은 성능

다음 노트북에서는 더 복잡한 **CIFAR-10** 데이터셋을 다룹니다.